In [1]:
import os.path as path
import raw_data_preprocessing as rdp
import warnings

warnings.filterwarnings('ignore')

data_folder_loc = '/dfs6/pub/yif12'
file_name = 'saved_data/April/kmeans_binary2/'
data_path = path.join(data_folder_loc, file_name)

import classification as clf
X, Y, feature_names, x_test, y_test = clf.read_inputs(data_path,['train_x.array','train_y.array','feature_names.list', 'test_x.array', 'test_y.array'])
print('The input and output shape: {} and {}'.format(X.shape, Y.shape))

The input and output shape: (684, 24) and (684,)


In [7]:
orig_len = 342

In [8]:
# 1) exlude game information
X = X[:,5:]
x_test = x_test[:,5:]
feature_names = feature_names[5:]

In [9]:
classes = ['long','short']
classify_test = clf.ClassificationProcessor(X, Y, orig_len, feature_names, classes)

In [ ]:
x_selected_train, x_selected_test, selected_feature_names = classify_test.feature_selection(x_test)

In [ ]:
selected_feature_names

array(['MR_accpre', 'IC_Zreverse_pre', 'NT_Zpre',
       'NEOFFIEmotionalstability', 'NEOFFIIntellectorOpenness',
       'VGQHoursCategorySumPastYear_Z', 'Psychohealth'], dtype=object)

In [ ]:
selected_clf = clf.ClassificationProcessor(x_selected_train, Y, orig_len, selected_feature_names, classes)

In [ ]:
selected_clf.Y

In [ ]:
selected_clf.model_selection(7)

## Final model

In [ ]:
from sklearn.svm import SVC, LinearSVC

c = [0.001, 0.01, 0.1,1]
tol = [1e-3, 1e-2, 1e-4]
shrinking = [False, True]
gamma = ["scale", "auto"]
for i in c:
    final_model = SVC(
        kernel="rbf",
        C=i,
        tol=0.001,
        gamma="scale",
        class_weight='balanced',
        random_state=0,
        probability=True,
    )
    selected_clf.final_model(final_model, x_selected_test, y_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

n_estimators = [300, 150, 100, 50]
criterion = ["gini", "entropy"]
max_depth = [3, 5, 7, None]
min_samples_split = [2, 3,4,5, 10]
min_samples_leaf = [2, 3,4,5, 10]
max_features = ["sqrt", "auto", "log2"]
for i in n_estimators:
    final_model = RandomForestClassifier(
        n_estimators=100,
        criterion="gini",
        max_depth=None,
        min_samples_split=4,
        min_samples_leaf=2,
        max_features="auto",
        class_weight="balanced",
        random_state=0,
    )
    selected_clf.final_model(final_model, x_selected_test, y_test)

In [ ]:
from sklearn.svm import SVC, LinearSVC

final_model = SVC(
    kernel="rbf",
    C=1,
    tol=0.001,
    gamma="scale",
    class_weight="balanced",
    random_state=0,
    probability=True,
)
final_model.fit(x_selected_train, Y)
y_pred = final_model.predict(x_selected_test)
y_proba = final_model.predict_proba(x_selected_test)

In [ ]:
selected_clf.shap_tree_cross('Random Forest',data_path)

shap

In [2]:
shap_all, expected_shap = clf.read_inputs(data_path,['shap_values.data','shap_expected.data'])
# selected_clf.shap_visualize(shap_all, expected_shap, selected_feature_names)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=5c9e1ba2-0cb9-40ed-a01b-b56de82eb464' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>